In [49]:
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
import numpy as np
import json
import os

In [2]:

gdf : gpd.GeoDataFrame = gpd.read_file('local/MB_2021_AUST_SHP_GDA2020/MB_2021_AUST_GDA2020.shp')
# 4m - 9m

In [3]:
gdf : gpd.GeoDataFrame = gdf[['MB_CODE21', 'geometry']].rename(columns={'MB_CODE21': 'id'})

In [11]:
gdfn : gpd.GeoDataFrame = gdf.dropna().copy()
# 10s

In [6]:
gdf_geom : gpd.GeoSeries = gdfn.boundary
# 30s

In [8]:
# Combine multiple linestring and multilinestring geometries into single multilinestring geometry
gdml = gdf_geom.apply(lambda x: geom.MultiLineString([x]) if isinstance(x, geom.LineString) else x)
# 1m
# 30s

In [14]:
gdfn['multilinestring'] = gdml

In [16]:
gdfn['line_order'] = gdfn['multilinestring'].apply(lambda x: np.array(range(len(x.geoms))))
# 15s

In [18]:
gdfn['linestring'] = gdfn['multilinestring'].apply(lambda x: x.geoms)
# 5s

In [26]:
gd_line_order = gdfn[['id', 'line_order']].explode('line_order')
# 1m 30s

In [28]:
gd_linestring = gdfn['linestring'].explode()
# 1m

In [33]:
df_lines = pd.concat([gd_line_order, gd_linestring], axis=1)

In [35]:
df_lines['linestring'] = df_lines['linestring'].apply(lambda x: np.array(x.coords))
# 30s - 1m

In [39]:
df_points = df_lines.explode('linestring')
# 4m

In [41]:
# Split the linestring into lon and lat
df_points['lon'] = df_points['linestring'].apply(lambda x: x[0])
df_points['lat'] = df_points['linestring'].apply(lambda x: x[1])
# 3m - 4m

In [45]:
df_point_order = df_lines['linestring'].apply(lambda x: np.array(range(len(x)))).explode()
# 20s

In [47]:
df_points['point_order'] = df_point_order
# 10s

In [50]:
# df_points.drop(columns=['linestring'], inplace=True)
# # 2m 30s

In [78]:
df_points = df_points[['id', 'line_order', 'point_order', 'lon', 'lat']]
# 30s

In [66]:
os.makedirs('data/mb-points', exist_ok=True)

In [80]:
df_points['id5'] = df_points['id'].str[:5]
# 30s - 1m30s

In [81]:
df_points['id5'].value_counts()

id5
11205    652163
11204    489706
30563    461821
20631    367844
50391    312449
          ...  
85830         6
85742         5
85839         5
85871         5
50172         5
Name: count, Length: 4225, dtype: int64

In [82]:
df_points.groupby('id5').apply(lambda x: x[['id', 'line_order', 'point_order', 'lon', 'lat']].to_csv(f'data/mb-points/mb-{x.name}.csv', index=False))
# 7m -8m

""


In [ ]:
# # Save the data into chunks of csv
# n = 200
# size = len(df_points)
# step = size // n
# for i in range(n):
#     start = i * step
#     end = (i + 1) * step
#     if i == n - 1:
#         end = size
#     df_points[start:end].to_csv(f'data/mb-points/mb-points-{i:03}.csv', index=False)

In [55]:
# # Split df_points into chunks by the first 2 characters of the id
# df_points['id2'] = df_points['id'].str[:2]
# # 30s - 1m

C:\Users\An\AppData\Local\Temp\ipykernel_22368\1883261777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_points['id2'] = df_points['id'].str[:2]


In [61]:
df_points

,id,line_order,point_order,lon,lat,id2
0,10000010000,0,0,146.927851,-36.083648,10
0,10000010000,0,1,146.928145,-36.082104,10
0,10000010000,0,2,146.928340,-36.081122,10
0,10000010000,0,3,146.929139,-36.081218,10
0,10000010000,0,4,146.929059,-36.081643,10
...,...,...,...,...,...,...
368283,97000003777,0,0,167.972812,-29.016558,97
368283,97000003777,0,1,167.972812,-29.015658,97
368283,97000003777,0,2,167.973840,-29.015658,97
368283,97000003777,0,3,167.973840,-29.016558,97
